# Python Pandas vs. SQL

http://pandas.pydata.org/pandas-docs/stable/comparison_with_sql.html

This page is meant to provide some examples of how various SQL operations would be performed using pandas.

- SELECT
- WHERE
- GROUP BY
- JOIN
- UNION
- UPDATE
- DELETE

In [1]:
import numpy as np
import pandas as pd

Most of the examples will utilize the tips dataset found within pandas tests. We’ll read the data into a DataFrame called tips and assume we have a database table of the same name and structure.

In [2]:
tips = pd.read_csv("data/tips.csv")

print tips.columns
print tips.index
print tips.shape

tips.head(5)

Index([u'total_bill', u'tip', u'sex', u'smoker', u'day', u'time', u'size'], dtype='object')
RangeIndex(start=0, stop=244, step=1)
(244, 7)


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


## SELECT

**SQL**

```SQL
SELECT total_bill, tip, smoker, time
FROM tips
LIMIT 5;
```

With pandas, column selection is done by passing a list of column names to your DataFrame:

In [3]:
tips[['total_bill', 'tip', 'smoker', 'time']].head(5)

,total_bill,tip,smoker,time
0,16.99,1.01,No,Dinner
1,10.34,1.66,No,Dinner
2,21.01,3.50,No,Dinner
3,23.68,3.31,No,Dinner
4,24.59,3.61,No,Dinner


## WHERE

```SQL
SELECT *
FROM tips
WHERE time = 'Dinner'
LIMIT 5;
```
DataFrames can be filtered in multiple ways; the most intuitive of which is using boolean indexing.

In [4]:
tips[tips.time == 'Dinner'].head(5)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


The above statement is simply passing a Series of True/False objects to the DataFrame, returning all rows with True.

Just like SQL’s OR and AND, multiple conditions can be passed to a DataFrame using **|** (OR) and **&** (AND).

In [5]:
tips[(tips['time'] == 'Dinner') & (tips.tip > 5.00)]

,total_bill,tip,sex,smoker,day,time,size
23,39.42,7.58,Male,No,Sat,Dinner,4
44,30.40,5.60,Male,No,Sun,Dinner,4
47,32.40,6.00,Male,No,Sun,Dinner,4
52,34.81,5.20,Female,No,Sun,Dinner,4
59,48.27,6.73,Male,No,Sat,Dinner,4
116,29.93,5.07,Male,No,Sun,Dinner,4
155,29.85,5.14,Female,No,Sun,Dinner,5
170,50.81,10.00,Male,Yes,Sat,Dinner,3
172,7.25,5.15,Male,Yes,Sun,Dinner,2
181,23.33,5.65,Male,Yes,Sun,Dinner,2


In [6]:
tips[(tips['size'] >= 5) | (tips['total_bill'] > 45)]

,total_bill,tip,sex,smoker,day,time,size
59,48.27,6.73,Male,No,Sat,Dinner,4
125,29.80,4.20,Female,No,Thur,Lunch,6
141,34.30,6.70,Male,No,Thur,Lunch,6
142,41.19,5.00,Male,No,Thur,Lunch,5
143,27.05,5.00,Female,No,Thur,Lunch,6
155,29.85,5.14,Female,No,Sun,Dinner,5
156,48.17,5.00,Male,No,Sun,Dinner,6
170,50.81,10.00,Male,Yes,Sat,Dinner,3
182,45.35,3.50,Male,Yes,Sun,Dinner,3
185,20.69,5.00,Male,No,Sun,Dinner,5


**NULL checking** is done using the notnull() and isnull() methods.

In [7]:
frame = pd.DataFrame({'col1': ['A', 'B', np.NaN, 'C', 'D'],
                      'col2': ['F', np.NaN, 'G', 'H', 'I']})

frame

,col1,col2
0,A,F
1,B,NaN
2,NaN,G
3,C,H
4,D,I


In [8]:
frame[frame['col2'].isnull()]

,col1,col2
1,B,NaN


In [9]:
frame[frame.col2.notnull() & frame['col1'].notnull()]

,col1,col2
0,A,F
3,C,H
4,D,I


## GROUP BY

In pandas, SQL's GROUP BY operations are performed using the similarly named method `groupby()`. This method typically refers to a process whee we'd like to split a dataset into grouops, apply some function (typically aggregation), and then combine the groups together.

A common SQL operation would be getting the count of records in each group throughout a dataset. For example, a query getting us the number of tips left by sex:

```SQL
SELECT sex, count(*)
FROM tips
GROUP BY sex;
```

The pandas equivalent would be:

In [10]:
tips.groupby('sex').size()

sex
Female     87
Male      157
dtype: int64

Notice that in pandas code we used `size()` and not `count()`. This is because `count()` applies the function to each column, returning the number of **`not null`** records within each.

In [11]:
tips.groupby('sex').count()

,total_bill,tip,smoker,day,time,size
sex,,,,,,
Female,87,87,87,87,87,87
Male,157,157,157,157,157,157


Alternatively, we could have applied the `count()` method to an individual column:

In [12]:
tips.groupby('sex')['total_bill'].count()

# or
# tips.groupby('sex').count()['total_bill']

sex
Female     87
Male      157
Name: total_bill, dtype: int64

Multiple functions can also be applied at once. For example, say we'd like to see how tip amount differs by day of the week - **`agg()`** allows you to pass a dictionary to your grouped DataFrame, indicating which functions to apply to specific columns.

```SQL
SELECT day, AVG(tip), COUNT(*)
FROM tips
GROUP BY day;
```

In [13]:
tips.groupby('day').agg({'tip':np.mean, 'day':np.size})

,tip,day
day,,
Fri,2.734737,19
Sat,2.993103,87
Sun,3.255132,76
Thur,2.771452,62


Grouping by more than one column is done by passing a list of columns to the **`groupby()`** method.

```SQL
SELECT smoker, day, COUNT(*), AVG(tip)
FROM tips
GROUP BY smoker, day;
```

In [14]:
tips.groupby(['smoker', 'day']).agg({'tip':[np.size, np.mean]})

tip          
             size      mean
smoker day                 
No     Fri    4.0  2.812500
       Sat   45.0  3.102889
       Sun   57.0  3.167895
       Thur  45.0  2.673778
Yes    Fri   15.0  2.714000
       Sat   42.0  2.875476
       Sun   19.0  3.516842
       Thur  17.0  3.030000

## JOIN

JOINs can be performed with `join()` or `merge()`. By default, `join()` will join the DataFrames on their indices. Each method has parameters allowing you to specify the type of join to perform (LEFT, RIGHT, INNER, FULL) or the columns to join on (column names or indices).

In [15]:
df1 = pd.DataFrame({'key':['A','B','C', 'D'],
                    'value':np.random.randn(4)})

df2 = pd.DataFrame({'key':['B','D','D', 'E'],
                    'value':np.random.randn(4)})
print df1
print df2

  key     value
0   A  0.299668
1   B -0.169282
2   C -0.007358
3   D  0.749899
  key     value
0   B -0.299071
1   D  1.460143
2   D  2.108179
3   E -0.070380


### INNER JOIN

```SQL
SELECT *
FROM df1
INNER JOIN df2
  ON df1.key = df2.key;
```

In [16]:
pd.merge(df1, df2, on='key')

,key,value_x,value_y
0,B,-0.169282,-0.299071
1,D,0.749899,1.460143
2,D,0.749899,2.108179


`merge()` also offers parameters for cases when you'd like to join one DataFrame's column with another DataFrame's index

In [17]:
indexed_df2 = df2.set_index('key')

pd.merge(df1, indexed_df2, left_on='key', right_index=True)

,key,value_x,value_y
1,B,-0.169282,-0.299071
3,D,0.749899,1.460143
3,D,0.749899,2.108179


### LEFT OUTER JOIN

```SQL
-- show all records from df1
SELECT *
FROM df1
LEFT OUTER JOIN df2
  ON df1.key = df2.key;
```

In [18]:
pd.merge(df1, df2, on='key', how='left')

,key,value_x,value_y
0,A,0.299668,NaN
1,B,-0.169282,-0.299071
2,C,-0.007358,NaN
3,D,0.749899,1.460143
4,D,0.749899,2.108179


### RIGHT JOIN

```SQL
-- show all records from df2
SELECT *
FROM df1
RIGHT OUTER JOIN df2
  ON df1.key = df2.key;
```

In [19]:
pd.merge(df1, df2, on='key', how='right')

,key,value_x,value_y
0,B,-0.169282,-0.299071
1,D,0.749899,1.460143
2,D,0.749899,2.108179
3,E,NaN,-0.070380


### FULL JOIN

Pandas also allows for FULL JOINs, which display both sides of the dataset, whether or not the joined columns find a match. As of writing, FULL JOINs are not supported in all RDBMS (MySQL)

```SQL
-- show all records from both tables
SELECT *
FROM df1
FULL OUTER JOIN df2
  ON df1.key = df2.key;
```

In [20]:
pd.merge(df1, df2, on='key', how='outer')

,key,value_x,value_y
0,A,0.299668,NaN
1,B,-0.169282,-0.299071
2,C,-0.007358,NaN
3,D,0.749899,1.460143
4,D,0.749899,2.108179
5,E,NaN,-0.070380


## UNION

UNION ALL can be performed using **`concat()`** function or **`df.append()`** method.

In [21]:
df1 = pd.DataFrame({'city': ['Chicago', 'San Francisco', 'New York City'],
                    'rank': range(1, 4)})

df2 = pd.DataFrame({'city': ['Chicago', 'Boston', 'Los Angeles'],
                    'rank': [1, 4, 5]})

print df1
print df2

            city  rank
0        Chicago     1
1  San Francisco     2
2  New York City     3
          city  rank
0      Chicago     1
1       Boston     4
2  Los Angeles     5


**SQL**

```SQL
SELECT city, rank
FROM df1
UNION ALL
SELECT city, rank
FROM df2;
/*
         city  rank
      Chicago     1
San Francisco     2
New York City     3
      Chicago     1
       Boston     4
  Los Angeles     5
*/
```

In [22]:
pd.concat([df1, df2])

,city,rank
0,Chicago,1
1,San Francisco,2
2,New York City,3
0,Chicago,1
1,Boston,4
2,Los Angeles,5


or **`df.append()`** method:

In [23]:
df1.append(df2)

,city,rank
0,Chicago,1
1,San Francisco,2
2,New York City,3
0,Chicago,1
1,Boston,4
2,Los Angeles,5


SQL's UNION is similar to UNION ALL, however UNION will remove duplicate rows

```SQL
SELECT city, rank
FROM df1
UNION
SELECT city, rank
FROM df2;
-- notice that there is only one Chicago record this time
/*
         city  rank
      Chicago     1
San Francisco     2
New York City     3
       Boston     4
  Los Angeles     5
*/
```

In pandas, we can use **`concat()`** or **`df.append()`** in conjunction with **`drop_duplicates()`**.

In [24]:
pd.concat([df1, df2]).drop_duplicates()

,city,rank
0,Chicago,1
1,San Francisco,2
2,New York City,3
1,Boston,4
2,Los Angeles,5


In [25]:
df1.append(df2).drop_duplicates()

,city,rank
0,Chicago,1
1,San Francisco,2
2,New York City,3
1,Boston,4
2,Los Angeles,5


## UPDATE

```SQL
UPDATE tips
SET tip = tip*2
WHERE tip < 2;
```

In [26]:
tips.loc[tips['tip'] < 2, 'tip'] *= 2

## DELETE

```SQL
DELETE FROM tips
WHERE tip > 9;
```

In pandas we select the rows that should remain, instead of deleting them

In [27]:
tips = tips.loc[tips['tip'] <= 9]